# Tensorflow Tutorial

## A. Graph

In [1]:
import tensorflow as tf
import numpy as np

상수를 더하기 예제

In [2]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)

total = a + b

# tensorboard를 위해 그래프 추가
# tensorboard 실행은 tensorboard --logdir=
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

with tf.Session() as sess:
    print(sess.run({'ab':(a, b), 'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


그래프의 동작 방식을 이해하기 위한 예제이다. 다음은 그래프가 3번 실행 된다.
마지막 실행에서 out1, out2가 동일한 vec값을 사용했다는 점을 확인하기 바란다.

In [ ]:
#변수 3개 반환
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2

with tf.Session() as sess:
    print('vec: {}'.format(sess.run(vec)))
    print('vec: {}'.format(sess.run(vec)))
    # out1, out2가 동일한 vec을 사용했다
    print('out: {}'.format(sess.run((out1, out2))))

입력 값을 넣기 위해 placeholder를 추가한 내용이다.

In [ ]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

z = x + y
with tf.Session() as sess:
    print(sess.run(z, feed_dict={x: 3, y: 4.5}))
    print(sess.run(z, feed_dict={x: [1, 3], y:[2, 4]}))

## B. Dataset

In [ ]:
source = np.array([
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7],
])

slices = tf.data.Dataset.from_tensor_slices(my_data)
iterator = slices.make_one_shot_iterator()
next_item = iterator.get_next()

sess = tf.Session()

while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break
        

In [ ]:
my_data = tf.random_normal([10, 3])
slices = tf.data.Dataset.from_tensor_slices(my_data)
iterator = slices.make_initializable_iterator()
next_row = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer)
    
    i = 0
    while True:
        try:
            i+=1
            print(i, sess.run(next_row))
        except tf.errors.OutOfRangeError:
            break

In [ ]:
# 입력 소스로 부터 dataset을 구성
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4, 10], minval=0, maxval=100))
print(dataset1.output_types)
print(dataset1.output_shapes)

# 다른 입력 소스를 묶어서 dataset을 구성
dataset2 = tf.data.Dataset.from_tensor_slices(
    (tf.random_uniform([4]), tf.random_uniform([4, 100]))
)

print(dataset2.output_types)
print(dataset2.output_shapes)

# dataset을 묶어서 새로운 dataset을 구성
dataset3 = tf.data.Dataset.zip(
    (dataset1, dataset2)
)
print(dataset3.output_types)
print(dataset3.output_shapes)


In [ ]:
# 딕셔너리를 이용한 입력소스 지정 방법. key에 이름을 부여할 수 있음
sources = dict()

sources['a'] = tf.random_uniform([4])
sources['b'] = tf.random_uniform([4, 100])

dataset = tf.data.Dataset.from_tensor_slices(sources)

print(dataset.output_types)
print(dataset.output_shapes)


### Iterator

#### 1. one-shot iterator
* initialization이 필요 없다.

In [ ]:
dataset = tf.data.Dataset.range(10)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    for i in range(10):
      print(sess.run(next_element))

#### 2. initalizable iterator
* 사용전, initialization 필수
* tf.placeholder를 이용해서 동작을 지정할 있는 장점이 있다.

In [ ]:
max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)

iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={max_value: 20})
    for i in range(10):
        print(sess.run(next_element))
    
with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={max_value:5})
    for i in range(5):
        print(sess.run(next_element))


#### 3. reinitilizable iterator
* 여러 Data source로 부터 데이터를 가지고 올수 있다. 단, structure은 동일해야 한다.

In [ ]:
training_dataset = tf.data.Dataset.range(10).map(lambda x: x + tf.random_uniform([], -10, 10, tf.int64))
validation_dataset = tf.data.Dataset.range(5)

# reinitializable iterator의 경우는 structure로 정의된다.
# 다른 Data source라 할지라도 같은 데이터 형태를 가지기 때문에 둘 중 하나를 사용하면 된다.
iterator = tf.data.Iterator.from_structure(training_dataset.output_types, training_dataset.output_shapes)
next_element = iterator.get_next()

training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

with tf.Session() as sess:
    
    #The number of epochs
    for e in range(20):
        print(f'> epoch {e}')
        
        # train sequence가 초기화 된다.
        sess.run(training_init_op)
        for _ in range(10):
            print(f'train: {sess.run(next_element)}')
        # validation sequence가 초기화 된다.
        sess.run(validation_init_op)
        for _ in range(5):
            print(f'valid: {sess.run(next_element)}')

#### 4. feedable iterator
* reinitializable iterator와 동일 기능 제공

In [ ]:
# repeat함수를 이용해서 반복되는 sequence를 생성할 수 있다.
# repeat함수를 사용하지 않으면 범위를 벗어날 경우, OutOfRangeError가 발생한다.
training_dataset = tf.data.Dataset.range(10).repeat()
validation_dataset = tf.data.Dataset.range(5)

handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(handle, 
                                               validation_dataset.output_types, 
                                               validation_dataset.output_shapes)

next_element = iterator.get_next()

training_iterator = training_dataset.make_one_shot_iterator()
validation_iterator = validation_dataset.make_initializable_iterator()


with tf.Session() as sess:
    # input 소스를 선택할 수 있는 핸들러
    training_handle = sess.run(training_iterator.string_handle())
    validation_handle = sess.run(validation_iterator.string_handle())
    
    while True:
        for _ in range(200):
            print('train ', sess.run(next_element, feed_dict={handle: training_handle}))

        sess.run(validation_iterator.initializer)
        for _ in range(5):
            print('valid ', sess.run(next_element, feed_dict={handle: validation_handle}))

In [ ]:
dataset = tf.data.Dataset.range(3)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()
result = tf.add(next_element, next_element)

with tf.Session() as sess:
    sess.run(iterator.initializer)
   
    # 일반적으로 train loop에 사용하는 try-except 블록
    try:
        print(sess.run(result)) # 0
        print(sess.run(result)) # 1
        print(sess.run(result)) # 2
    except tf.errors.OutOfRangeError:
        print('End of dataset')

In [ ]:
dataset1 = tf.data.Dataset.from_tensor_slices(['A', 'B', 'C'])
dataset2 = tf.data.Dataset.from_tensor_slices(([1, 2, 3], ['A1', 'B1', 'C1']))
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))

iterator = dataset3.make_initializable_iterator()
next1, (next2, next3) = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer)
    print(sess.run((next1, (next2, next3))))
    # iterator에 묶여 있어서 동시에 다음 element로 넘어간다.
    print(sess.run(next1)) 
    print(sess.run((next1, (next2, next3))))
    
    

### C. Reading input data

#### 1. Consuming NumPy arrays

In [ ]:
x = np.array([[1, 2, 3], [4, 5, 6]])
y = np.array([0, 1])

np.savez('./array', features=x, labels=y)
data = np.load('array.npz')

features = data['features']
labels = data['labels']

# The length of features and labels should be the same.
assert features.shape[0] == labels.shape[0]

feature_placeholder = tf.placeholder(features.dtype, features.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

dataset = tf.data.Dataset.from_tensor_slices((feature_placeholder, labels_placeholder))
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={feature_placeholder: features, labels_placeholder: labels})
    print(sess.run(next_element))
    print(sess.run(next_element))

#### 2. Consuming TFRecord data

#### 3. Consuming text data

### D. Preprocessing data with Dataset.map()

#### 1. Parsing tf.Example protocol buffer messages

#### 2. Decoding image data and resizing it

#### 3. Applying arbitrary Python logic with tf.py_func()

### E. Batching dataset elements

#### 1. Simple batching

In [ ]:
inc_dataset = tf.data.Dataset.range(100)
dec_dataset = tf.data.Dataset.range(0, -100, -1)
dataset = tf.data.Dataset.zip((inc_dataset, dec_dataset))
# basch 사이즈 지정

batched_dataset = dataset.batch(4)

iterator = batched_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    print(sess.run(next_element))  # ==> ([0, 1, 2,   3],   [ 0, -1,  -2,  -3])
    print(sess.run(next_element))  # ==> ([4, 5, 6,   7],   [-4, -5,  -6,  -7])
    print(sess.run(next_element))  # ==> ([8, 9, 10, 11],   [-8, -9, -10, -11])

#### 2. Batching tensors with padding

## Layers